# Experiments

This Jupyter Notebook presents different experiments done to videos. This experiments focus on the temporal consistency in cartoon-like videos. We focus on using the optical flow to see how consecutive frames change.

First, we import the functions needed from the notebook called "Functions".

In [20]:
#%run Functions.ipynb

In [1]:
import import_ipynb
from Functions import *

importing Jupyter notebook from Functions.ipynb


We load a video and store its frames in a list to then add occlusions to 3 consecutive frames.

In [2]:
cap = open_vid('VDB\L.mp4')
Frames = get_frames(cap)
_,_,fps,_ = get_props(cap)
F = Frames.copy()
loc=[[150,50]]
sizes = [[50,100]]
N = 15
F[N] = occlusions(F[N],loc=loc,sizes=sizes,shapes=['rectangle'], colors=(200,100,50))
F[N+1] = occlusions(F[N+1],loc=loc,sizes=sizes,shapes=['rectangle'], colors=(200,100,50))
F[N+2] = occlusions(F[N+2],loc=loc,sizes=sizes,shapes=['rectangle'], colors=(200,100,50))

Width:  640
Height:  360
FPS:  23.976023976023978
Frame Count:  2326


We play the video to see the occlusions.<br>
Use 'q' to stop and close window at any time.

In [3]:
play_frames(F[:30],fps)

Delay:  41


Then we calculate different properties in the series of frames.

First, we calculate and display the magnitude of the optical flow by using Furneback's method.<br>
Use the arrows to move forward or backward in the frame sequence.<br>
Use 'q' at any time to stop and close windows.

In [8]:
_ = OFM(F[:30])

We can also see the vector field of the optical flow in with the green arrows. Also, we see the sum of these vectors as the red arrow.

In [6]:
_ = OFV(F[:30])

We can also use the Lucas-Kanade method to calculate the optical flow.

In [7]:
_ = OFLK(F[:30])

And finally, we can compute the optical flow using Phase Correlation.

In [9]:
_ = PhaseC(F[:30])

Another way to se how consecutive frames change from one to another, we can calculate its difference.

First, we calculate the difference between a pair of consecutive frames and use a threshold to create a binary image that shows the region with most changes.

In [10]:
_ = frame_dif(F[:30])

But for more simplicity, we can just calculate the differences and normalize them.

In [11]:
_ = frame_dif1(F[:30])

Now, lets try to cartoonize an image.

In [2]:
file = 'IDB/therock.jpg'
img = cv2.imread(file)
toon = cartoonize_image(img,kc=3)
display_images([img,toon])

What about cartoonizing a video?<br>
To speed up the cartoonization, the frames in the video are resized to smaller ones and then returned to their original size. Take in consideration that this will also degrade the quality of the cartoonized video.

In [35]:
cap = open_vid('VDB/U.mp4')
Frames = get_frames(cap)
w,h,fps,_ = get_props(cap)
Toon = [cv2.resize(cartoonize_image(cv2.resize(f,(200,200),interpolation=cv2.INTER_AREA)),(w,h),interpolation=cv2.INTER_CUBIC) for f in Frames]
#save_vid(Toon,'Cartoonized/U_toonzz.mp4',fps)

Width:  640
Height:  320
FPS:  24.0
Frame Count:  69


Lets see the original video.

In [33]:
play_frames(Frames,fps)

Delay:  41


Lets see the cartoonized video now.

In [36]:
play_frames(Toon,fps)

Delay:  41


Now lets see how the optical flow behave for the original video and the cartoonized video.

In [4]:
cap = open_vid('VDB/U.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
Frames = get_frames(cap)
cap = open_vid('Cartoonized/U_toon.mp4')
Toon = get_frames(cap)
play_frames(Frames,fps)
play_frames(Toon,fps)
_ = OFV(Frames,20)
_ = OFV(Toon,20)

Delay:  41
Delay:  41


To have more control over the optical flow, lets capture the video frames from the webcam and display it in real time.

In [11]:
cap_of()

We can compute different variability indexes of the pixel of an image. Some of these are calculated below as the relative luminance, the color variace and a variability index based on neighboring pixels. The greater the variability index, the more variability there is.

In [10]:
file = 'IDB/party.jpg'
img = cv2.imread(file)
print(relative_luminance(img))
print(color_variance(img))
print(calculate_variability(img))
display_frame(img)

0.5007049189404322
0.003574134719467966
0.7609107919731268


In [5]:
#Entorno/Interfaz